The metrics functions expect `predictions` to be a `pandas.DataFrame`:

* Index: `target_datetime_utc`
* Columns:
  - `t0_datetime_utc`
  - `forecast_pv_yield`  # Values must be in the range [0, 1]

The `actual` values are a `pd.Series`, where the index is the `target_datetime_utc`,
and the values are the `actual_pv_yield` (in the range [0, 1]).

Metrics are defined by a `metrics_pipeline` which is a tuple of one or more `callable` objects.
For example:

```python
# Define re-usable metrics pipelines: The keys of the dict are human-readable metric names,
# and the values of the dict are the pipeline tuples.
metrics_pipelines: dict[str, tuple[Callable, ...]] = {
    "MAE in MW, ignoring night": (
        # Multiply the `predictions` and the `actual` by national installed PV capacity:
        Denormalize(national_pv_capacity_mwp_per_target_datetime_utc),
        # Drop any values that occur at "night". Where "night" is defined as when the Sun
        # is below a particular angle in the sky:
        IgnoreNight(latitude, longitude, threshold_for_sun_angle_in_degrees=-5),
        # Compute the absolute error for each timestep:
        absolute_error,
        # Compute the mean across all timesteps:
        mean,
    ),
    "NMAE per month, including night": (
        # Compute the absolute error for each timestep:
        absolute_error,
        # Group by month:
        GroupBy(pd.Grouper(freq="M")),
        # Compute the mean absolute error per month:
        mean,
    ),
    "NMAE per hour, ignoring night": (
        IgnoreNight(latitude, longitude, threshold_for_sun_angle_in_degrees=-5),
        absolute_error,
        # Group by hour of day:
        GroupBy(pd.Grouper(freq="H")),
        # Compute the mean absolute error per hour of the day:
        mean,
    ),
    "NMAE per forecast step, ignoring night": (
        IgnoreNight(latitude, longitude, threshold_for_sun_angle_in_degrees=-5),
        absolute_error,
        # Group by forecast step:
        GroupBy(pd.Grouper(key="step")),
        # Compute the mean absolute error per hour of the day:
        mean,
    )
}

# Run the pipelines:
metrics_results: dict[str, Union[Number, pd.Series, pd.DataFrame]] = run_all_pipelines(
    metrics_pipelines, predictions, actual)
```

In [15]:
from pv_forecast.metrics import absolute_error
import pandas as pd
import numpy as np


target_datetime_utc = pd.date_range(
    start="2020-01-01", end="2020-01-10", freq="30T", tz="UTC", name="target_datetime_utc")
step = pd.timedelta_range(start="0 hours", end="48 hours", freq="30T", closed="left", name="step")
    

In [10]:
predictions = pd.Series(
    0,
    pd.MultiIndex.from_product([target_datetime_utc, step]),
    name="forecast_pv_yield",
)

In [11]:
predictions

target_datetime_utc        step           
2020-01-01 00:00:00+00:00  0 days 00:00:00    0
                           0 days 00:30:00    0
                           0 days 01:00:00    0
                           0 days 01:30:00    0
                           0 days 02:00:00    0
                                             ..
2020-01-10 00:00:00+00:00  1 days 21:30:00    0
                           1 days 22:00:00    0
                           1 days 22:30:00    0
                           1 days 23:00:00    0
                           1 days 23:30:00    0
Name: forecast_pv_yield, Length: 41568, dtype: int64

In [12]:
actual = pd.Series(
    1,
    target_datetime_utc,
    name="actual_pv_yield",
)

In [25]:
pd.core.groupby.GroupBy.mean(np.abs(np.subtract(predictions, actual)).groupby(
    pd.Grouper(level="step", freq="3H")))

step
0 days 00:00:00    1.0
0 days 03:00:00    1.0
0 days 06:00:00    1.0
0 days 09:00:00    1.0
0 days 12:00:00    1.0
0 days 15:00:00    1.0
0 days 18:00:00    1.0
0 days 21:00:00    1.0
1 days 00:00:00    1.0
1 days 03:00:00    1.0
1 days 06:00:00    1.0
1 days 09:00:00    1.0
1 days 12:00:00    1.0
1 days 15:00:00    1.0
1 days 18:00:00    1.0
1 days 21:00:00    1.0
Freq: 3H, dtype: float64

<function pandas.core.groupby.groupby.GroupBy.mean(self, numeric_only: 'bool | lib.NoDefault' = <no_default>, engine: 'str' = 'cython', engine_kwargs: 'dict[str, bool] | None' = None)>

In [26]:
pd.Series.groupby

<function pandas.core.series.Series.groupby(self, by=None, axis: 'Axis' = 0, level: 'Level' = None, as_index: 'bool' = True, sort: 'bool' = True, group_keys: 'bool | lib.NoDefault' = <no_default>, squeeze: 'bool | lib.NoDefault' = <no_default>, observed: 'bool' = False, dropna: 'bool' = True) -> 'SeriesGroupBy'>